In [1]:
import re

response = "The row numbers in the metadata sheet which have the Visual_Type 'barChart' are: 32, 31, 2"
rows= re.findall(r'\b\d+\b', response)
rows_list= [int(num) for num in rows]
print(rows_list)

[32, 31, 2]


In [2]:
x = [1,2,3]

In [4]:
type(x[0])

int

In [23]:
import pandas as pd
combined_df = pd.read_csv("Merged_MasterData.csv")

new_df = combined_df.sample(5)
print(new_df.index)

Int64Index([13, 15, 5, 18, 1], dtype='int64')


In [28]:
new_df

,Report_Name,Page_Name,Visual_Title,Visual_Type,Columns,Description
13,POC_Corporate_Spend,IT Spend Trend,Slicer,slicer,Sales Region,The Power BI visual Slicer is a type of slicer...
15,POC_Corporate_Spend,IT Spend Trend,Decomposition tree,decompositionTreeVisual,"Var Plan, Sales Region, Period, IT Area, IT Su...",The Decomposition Tree visual in Power BI is a...
5,POC_Regional_Sales,Sales Overview,Forecast by Territory,pivotTable,"Territory, State or Province, Revenue Won, In ...",The Power BI visual Forecast by Territory is a...
18,POC_Corporate_Spend,Plan Variance Analysis,Slicer,slicer,Var Plan %,The Power BI visual Slicer is a type of slicer...
1,POC_Regional_Sales,Sales Overview,Forecast by Location,shapeMap,"State or Province, State or Province.1, Rev Go...",The Power BI visual Forecast by Location is a ...


In [30]:
ls = [13, 15, 5]
new_df.loc[ls]

,Report_Name,Page_Name,Visual_Title,Visual_Type,Columns,Description
13,POC_Corporate_Spend,IT Spend Trend,Slicer,slicer,Sales Region,The Power BI visual Slicer is a type of slicer...
15,POC_Corporate_Spend,IT Spend Trend,Decomposition tree,decompositionTreeVisual,"Var Plan, Sales Region, Period, IT Area, IT Su...",The Decomposition Tree visual in Power BI is a...
5,POC_Regional_Sales,Sales Overview,Forecast by Territory,pivotTable,"Territory, State or Province, Revenue Won, In ...",The Power BI visual Forecast by Territory is a...


In [104]:
# %pip install openai
# %pip install semantic-link
# %pip install langchain
# %pip install pandasai
# %pip install faiss-cpu

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import string
import gensim
import operator
import re

In [3]:
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.vectorstores.faiss import FAISS
# from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
# from langchain.chat_models import AzureChatOpenAI

In [106]:
import os
import openai
from openai import AzureOpenAI
import pandas as pd
from IPython.display import display

In [107]:
os.environ["OPENAI_API_TYPE"] = "azure"
 
os.environ["OPENAI_API_KEY"] = "6ad08c8e58dd4de9985b86b1209d9cc2"
 
os.environ["OPENAI_API_BASE"] = "https://azureopenaitext.openai.azure.com/"
 
os.environ["OPENAI_API_VERSION"] = "2023-09-15-preview"

In [108]:
openai.api_type = "azure"
openai.azure_endpoint = "https://azureopenaitext.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = "6ad08c8e58dd4de9985b86b1209d9cc2"

In [109]:
def create_list_of_strings_from_a_DF(input_df):
    combined_strings = []

    for index, row in input_df.iterrows():
        combined_string = '-'.join(row.astype(str))  # Combine values in the row with '-'
        combined_strings.append(combined_string)

    #print(combined_strings)
    return combined_string

def create_list_of_strings_of_a_column(df_column):
    combined_strings = [f"{index}: {value}" for index, value in enumerate(df_column)]#, start=1)]
    #combined_strings = df_column.tolist()
    return combined_strings

In [110]:
combined_df = pd.read_csv(r'PowerBI_Report_Metadata\multiple_reports.csv',encoding="ISO-8859-1")
# combined_df

In [111]:
def result_df(result):
    data_dict = {'Index': [], 'Score': []}  # Initialize an empty dictionary to store data

    # Assuming 'results' contains the list of tuples you've provided

    for item in result:
        # Extract index number and score
        index = int(item[0].page_content.split(':')[0])
        score = item[1]
        
        # Append index number and score to the dictionary
        data_dict['Index'].append(index)
        data_dict['Score'].append(score)
        
    scores_df = pd.DataFrame(data_dict)
    scores_df = scores_df.sort_values('Index')
    return scores_df

In [112]:
embeddings = AzureOpenAIEmbeddings()#OpenAIEmbeddings()

c:\Program Files\Python310\lib\site-packages\langchain\embeddings\azure_openai.py:105: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://azureopenaitext.openai.azure.com/ to https://azureopenaitext.openai.azure.com//openai.
  warnings.warn(


In [113]:
embeddings_input = create_list_of_strings_of_a_column(combined_df['Description'])
document_search = FAISS.from_texts(embeddings_input,embeddings)

In [62]:

# #def query_result(df):

df_list = []
    
query_list = ['Which report is using bar chart?',
            'Can you fetch me a report which has sales column used?',
            'Report with average revenue.',
            'Show me report with month wise revenue count.',
            'get a report which uses filter on product code.']



for query in query_list:
    
    # Create a new DataFrame with selected columns
    new_df = combined_df.copy()#[['Report_Name', 'Page_Name', 'Title']].copy()
    
    result = document_search.similarity_search_with_score(query,len(embeddings_input))
    new_df['Score'] = result_df(result)['Score'].values
    
    df_list.append(new_df)

In [63]:
context_df = df_list[0].nsmallest(7,'Score')
context_df

,Report_Name,Page_Name,Title,Type,Columns,Description,Score
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,barChart,"Region, State, Revenue","The Power BI visual titled ""Revenue by Region ...",0.354691
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,lineStackedColumnComboChart,"Month, Revenue, Opportunity Count","The Power BI visual titled ""Revenue and Opport...",0.368216
12,POC_Regional_Sales,Sales Overview,(Right click to drill to Users),lineStackedColumnComboChart,"Manager, Owner, Revenue won, Qualified pipelin...","The Power BI visual ""Right click to drill to U...",0.372951
26,POC_Corporate_Spend,IT Spend Trend,Actual and Plan by IT Area,lineStackedColumnComboChart,Sales Region,The Power BI visual Actual and Plan by IT Area...,0.377670
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,barChart,"Sales Stage, Opportunity Count","The Power BI visual titled ""Pipeline by Stage""...",0.379558
11,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I...",0.383969
23,POC_Corporate_Spend,IT Spend Trend,Actual by Period and Business Area,stackedAreaChart,"Period, Business Area, Actual","The Power BI visual ""Actual by Period and Busi...",0.390342


In [64]:
print(df_list[0]['Score'].values)

[0.43263128 0.43552542 0.44261885 0.42203116 0.42325383 0.36821586
 0.37955835 0.3546914  0.44998583 0.42844212 0.44809243 0.3839687
 0.37295055 0.4124266  0.443577   0.43129674 0.4562358  0.4615062
 0.42549932 0.458794   0.4059692  0.4540452  0.45733625 0.3903417
 0.45154476 0.4237912  0.37767    0.44059223 0.44668254 0.44199598
 0.433612   0.43934914 0.39212433 0.44901106]


In [81]:
client = AzureOpenAI(
        api_key = openai.api_key,  
        api_version = openai.api_version,
        azure_endpoint = openai.azure_endpoint
    )

In [96]:
def openai_response(prompt,table):
    response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant proficient in PowerBI. Answer the following query by referring to table given by providing the appropriate row number. Response should be strictly row numbers!"},
        {"role": "user", "content": f"{prompt}:{table}"}
        ],
    temperature=1
    )
    
    return response.choices[0].message.content

In [97]:
response_1 = openai_response(query_list[0],context_df.drop('Score',axis='columns'))
response_1

'Report row numbers using bar chart are: 7, 6, and 11.'

In [114]:
for query, dataframe in zip(query_list,df_list):
    response = openai_response(query, dataframe.nsmallest(10,'Score').drop('Score',axis='columns'))
    print('-----------------',query,'-----------------\n',response,'\n\n\n')
    display(dataframe.nsmallest(10,'Score'))

----------------- Which report is using bar chart? -----------------
 Report number 7 and 6 are using bar chart. 





,Report_Name,Page_Name,Title,Type,Columns,Description,Score
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,barChart,"Region, State, Revenue","The Power BI visual titled ""Revenue by Region ...",0.354691
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,lineStackedColumnComboChart,"Month, Revenue, Opportunity Count","The Power BI visual titled ""Revenue and Opport...",0.368216
12,POC_Regional_Sales,Sales Overview,(Right click to drill to Users),lineStackedColumnComboChart,"Manager, Owner, Revenue won, Qualified pipelin...","The Power BI visual ""Right click to drill to U...",0.372951
26,POC_Corporate_Spend,IT Spend Trend,Actual and Plan by IT Area,lineStackedColumnComboChart,Sales Region,The Power BI visual Actual and Plan by IT Area...,0.377670
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,barChart,"Sales Stage, Opportunity Count","The Power BI visual titled ""Pipeline by Stage""...",0.379558
11,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I...",0.383969
23,POC_Corporate_Spend,IT Spend Trend,Actual by Period and Business Area,stackedAreaChart,"Period, Business Area, Actual","The Power BI visual ""Actual by Period and Busi...",0.390342
32,POC_Corporate_Spend,Plan Variance Analysis,Var Plan by Country/Region,clusteredBarChart,"Business Area, Var Plan %",The Power BI visual Var Plan by Country/Region...,0.392124
20,POC_Regional_Sales,Sales Overview,Blank by Forecast Adjustment,columnChart,Forecast Adjustment,The Power BI visual Blank by Forecast Adjustme...,0.405969
13,POC_Regional_Sales,Sales Overview,Revenue and forecast by Product,pivotTable,"Product Category, Product, Revenue Won, Qualif...","The Power BI visual ""Revenue and forecast by P...",0.412427


----------------- Can you fetch me a report which has sales column used? -----------------
 The report which has sales column used are:
1. POC_Regional_Sales (row 13)
2. POC_Revenue Opportunities (row 5)
3. POC_Regional_Sales (row 18)
4. POC_Revenue Opportunities (row 6)
5. POC_Regional_Sales (row 9)
6. POC_Regional_Sales (row 21)
7. POC_Corporate_Spend (row 26)
8. POC_Regional_Sales (row 12)
9. POC_Revenue Opportunities (row 7)
10. POC_Revenue Opportunities (row 4) 





,Report_Name,Page_Name,Title,Type,Columns,Description,Score
13,POC_Regional_Sales,Sales Overview,Revenue and forecast by Product,pivotTable,"Product Category, Product, Revenue Won, Qualif...","The Power BI visual ""Revenue and forecast by P...",0.395207
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,lineStackedColumnComboChart,"Month, Revenue, Opportunity Count","The Power BI visual titled ""Revenue and Opport...",0.423760
18,POC_Regional_Sales,Sales Overview,Revenue won,kpi,"Revenue Won, Blank","The Power BI visual ""Revenue Won"" is a KPI (Ke...",0.427390
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,barChart,"Sales Stage, Opportunity Count","The Power BI visual titled ""Pipeline by Stage""...",0.428417
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,funnel,"Sales Stage, Revenue Open",The Revenue Open by Sales Stage visual in Powe...,0.429604
21,POC_Regional_Sales,Sales Overview,Forecast Adjustment,slicer,"State or Province, State or Province.1, Rev Go...",The Power BI visual Forecast Adjustment is a s...,0.433546
26,POC_Corporate_Spend,IT Spend Trend,Actual and Plan by IT Area,lineStackedColumnComboChart,Sales Region,The Power BI visual Actual and Plan by IT Area...,0.434065
12,POC_Regional_Sales,Sales Overview,(Right click to drill to Users),lineStackedColumnComboChart,"Manager, Owner, Revenue won, Qualified pipelin...","The Power BI visual ""Right click to drill to U...",0.434286
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,barChart,"Region, State, Revenue","The Power BI visual titled ""Revenue by Region ...",0.434311
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,filledMap,"State, Revenue","The Power BI visual titled ""Revenue by State"" ...",0.434831


----------------- Report with average revenue. -----------------
 Row numbers 0, 4, 5, 7, 8, 9, 11, 13, 17 and 18 have columns related to revenue. 





,Report_Name,Page_Name,Title,Type,Columns,Description,Score
18,POC_Regional_Sales,Sales Overview,Revenue won,kpi,"Revenue Won, Blank","The Power BI visual ""Revenue Won"" is a KPI (Ke...",0.369753
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,barChart,"Region, State, Revenue","The Power BI visual titled ""Revenue by Region ...",0.375318
13,POC_Regional_Sales,Sales Overview,Revenue and forecast by Product,pivotTable,"Product Category, Product, Revenue Won, Qualif...","The Power BI visual ""Revenue and forecast by P...",0.376759
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,filledMap,"State, Revenue","The Power BI visual titled ""Revenue by State"" ...",0.377505
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,card,Opportunity Revenue,"The Power BI visual titled ""Revenue"" is a Card...",0.377943
17,POC_Regional_Sales,Sales Overview,Revenue goal,kpi,"Rev Goal, Blank",The Revenue Goal visual in Power BI is a KPI (...,0.382058
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,lineStackedColumnComboChart,"Month, Revenue, Opportunity Count","The Power BI visual titled ""Revenue and Opport...",0.386277
11,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I...",0.387549
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,funnel,"Sales Stage, Revenue Open",The Revenue Open by Sales Stage visual in Powe...,0.390238
8,POC_Revenue Opportunities,Detail,Matrix,pivotTable,"Opportunity Size, Region, Partner, Opportuniti...","The Power BI visual titled ""Matrix"" is a dynam...",0.408784


----------------- Show me report with month wise revenue count. -----------------
 Row numbers for the report with month wise revenue count are: 5. 





,Report_Name,Page_Name,Title,Type,Columns,Description,Score
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,lineStackedColumnComboChart,"Month, Revenue, Opportunity Count","The Power BI visual titled ""Revenue and Opport...",0.289506
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,barChart,"Region, State, Revenue","The Power BI visual titled ""Revenue by Region ...",0.345317
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,filledMap,"State, Revenue","The Power BI visual titled ""Revenue by State"" ...",0.349079
18,POC_Regional_Sales,Sales Overview,Revenue won,kpi,"Revenue Won, Blank","The Power BI visual ""Revenue Won"" is a KPI (Ke...",0.356704
11,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I...",0.360073
13,POC_Regional_Sales,Sales Overview,Revenue and forecast by Product,pivotTable,"Product Category, Product, Revenue Won, Qualif...","The Power BI visual ""Revenue and forecast by P...",0.363440
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,funnel,"Sales Stage, Revenue Open",The Revenue Open by Sales Stage visual in Powe...,0.376540
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,card,Opportunity Revenue,"The Power BI visual titled ""Revenue"" is a Card...",0.380135
14,POC_Regional_Sales,Sales Overview,Forecast by Territory,pivotTable,"Territory, State or Province, Revenue Won, In ...",The Power BI visual Forecast by Territory is a...,0.390530
17,POC_Regional_Sales,Sales Overview,Revenue goal,kpi,"Rev Goal, Blank",The Revenue Goal visual in Power BI is a KPI (...,0.392707


----------------- get a report which uses filter on product code. -----------------
 Rows 2, 13, 11, 22, 27, 7, 21, and 26 have columns related to product code. 





,Report_Name,Page_Name,Title,Type,Columns,Description,Score
2,POC_Revenue Opportunities,Opportunity Overview,Product Slicer,slicer,Product Code,"The Power BI visual titled ""Product Slicer"" is...",0.411536
13,POC_Regional_Sales,Sales Overview,Revenue and forecast by Product,pivotTable,"Product Category, Product, Revenue Won, Qualif...","The Power BI visual ""Revenue and forecast by P...",0.466952
11,POC_Regional_Sales,Sales Overview,Revenue Won and Revenue In Pipeline by Product...,barChart,"Product Category, Product, Revenue Won, Revenu...","The Power BI visual ""Revenue Won and Revenue I...",0.490374
22,POC_Corporate_Spend,IT Spend Trend,Slicer,slicer,Sales Region,The Power BI visual Slicer is a type of slicer...,0.515235
27,POC_Corporate_Spend,Plan Variance Analysis,Slicer,slicer,Var Plan %,The Power BI visual Slicer is a type of slicer...,0.524843
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,barChart,"Sales Stage, Opportunity Count","The Power BI visual titled ""Pipeline by Stage""...",0.535480
12,POC_Regional_Sales,Sales Overview,(Right click to drill to Users),lineStackedColumnComboChart,"Manager, Owner, Revenue won, Qualified pipelin...","The Power BI visual ""Right click to drill to U...",0.538583
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,barChart,"Region, State, Revenue","The Power BI visual titled ""Revenue by Region ...",0.540165
21,POC_Regional_Sales,Sales Overview,Forecast Adjustment,slicer,"State or Province, State or Province.1, Rev Go...",The Power BI visual Forecast Adjustment is a s...,0.541806
26,POC_Corporate_Spend,IT Spend Trend,Actual and Plan by IT Area,lineStackedColumnComboChart,Sales Region,The Power BI visual Actual and Plan by IT Area...,0.543535


In [62]:
print('---------------------',query_list[0],'---------------------')
df_list[0]

--------------------- Which report is using bar chart? ---------------------


,Report_Name,Page_Name,Title,Score
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,0.432631
1,POC_Revenue Opportunities,Opportunity Overview,Count,0.435525
2,POC_Revenue Opportunities,Opportunity Overview,Product Slicer,0.442619
3,POC_Revenue Opportunities,Opportunity Overview,Opportunity Count by Region,0.422031
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,0.423254
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,0.368216
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,0.379558
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,0.354691
8,POC_Revenue Opportunities,Detail,Matrix,0.449986
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,0.428442


In [73]:
min_index = df_list[0]['Score'].idxmin()

row_with_lowest_score = df_list[0].loc[min_index]
print(row_with_lowest_score)

Report_Name      POC_Revenue Opportunities
Page_Name             Opportunity Overview
Title          Revenue by Region and State
Score                             0.354691
Name: 7, dtype: object


In [74]:
print('---------------------',query_list[1],'---------------------')
df_list[1]

--------------------- Can you fetch me a report which has sales column used? ---------------------


,Report_Name,Page_Name,Title,Score
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,0.449540
1,POC_Revenue Opportunities,Opportunity Overview,Count,0.499021
2,POC_Revenue Opportunities,Opportunity Overview,Product Slicer,0.447701
3,POC_Revenue Opportunities,Opportunity Overview,Opportunity Count by Region,0.483619
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,0.434831
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,0.423760
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,0.428417
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,0.434311
8,POC_Revenue Opportunities,Detail,Matrix,0.438520
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,0.429604


In [69]:
min_index = df_list[1]['Score'].idxmin()

# Access and print the row with the lowest 'Score' value
row_with_lowest_score = df_list[1].loc[min_index]
print(row_with_lowest_score)

Report_Name                 POC_Regional_Sales
Page_Name                       Sales Overview
Title          Revenue and forecast by Product
Score                                 0.395207
Name: 13, dtype: object


In [64]:
print('---------------------',query_list[2],'---------------------')
df_list[2]

--------------------- Report with average revenue. ---------------------


,Report_Name,Page_Name,Title,Score
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,0.377943
1,POC_Revenue Opportunities,Opportunity Overview,Count,0.501959
2,POC_Revenue Opportunities,Opportunity Overview,Product Slicer,0.542500
3,POC_Revenue Opportunities,Opportunity Overview,Opportunity Count by Region,0.475156
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,0.377505
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,0.386277
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,0.478860
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,0.375318
8,POC_Revenue Opportunities,Detail,Matrix,0.408784
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,0.390238


In [75]:
min_index = df_list[2]['Score'].idxmin()

# Access and print the row with the lowest 'Score' value
row_with_lowest_score = df_list[2].loc[min_index]
print(row_with_lowest_score)

Report_Name    POC_Regional_Sales
Page_Name          Sales Overview
Title                 Revenue won
Score                    0.369753
Name: 18, dtype: object


In [76]:
print('---------------------',query_list[3],'---------------------')
df_list[3]

--------------------- Show me report with month wise revenue count. ---------------------


,Report_Name,Page_Name,Title,Score
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,0.380135
1,POC_Revenue Opportunities,Opportunity Overview,Count,0.447772
2,POC_Revenue Opportunities,Opportunity Overview,Product Slicer,0.489512
3,POC_Revenue Opportunities,Opportunity Overview,Opportunity Count by Region,0.421621
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,0.349079
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,0.289506
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,0.438678
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,0.345317
8,POC_Revenue Opportunities,Detail,Matrix,0.425076
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,0.376540


In [71]:
min_index = df_list[3]['Score'].idxmin()

# Access and print the row with the lowest 'Score' value
row_with_lowest_score = df_list[3].loc[min_index]
print(row_with_lowest_score)

Report_Name                 POC_Revenue Opportunities
Page_Name                        Opportunity Overview
Title          Revenue and Opportunity Count by Month
Score                                        0.289506
Name: 5, dtype: object


In [66]:
print('---------------------',query_list[4],'---------------------')
df_list[4]

--------------------- get a report which uses filter on product code. ---------------------


,Report_Name,Page_Name,Title,Score
0,POC_Revenue Opportunities,Opportunity Overview,Revenue,0.568053
1,POC_Revenue Opportunities,Opportunity Overview,Count,0.567254
2,POC_Revenue Opportunities,Opportunity Overview,Product Slicer,0.411536
3,POC_Revenue Opportunities,Opportunity Overview,Opportunity Count by Region,0.563725
4,POC_Revenue Opportunities,Opportunity Overview,Revenue by State,0.556004
5,POC_Revenue Opportunities,Opportunity Overview,Revenue and Opportunity Count by Month,0.550527
6,POC_Revenue Opportunities,Opportunity Overview,Pipeline by Stage,0.535480
7,POC_Revenue Opportunities,Opportunity Overview,Revenue by Region and State,0.540165
8,POC_Revenue Opportunities,Detail,Matrix,0.555219
9,POC_Regional_Sales,Sales Overview,Revenue Open by Sales Stage,0.564136


In [72]:
min_index = df_list[4]['Score'].idxmin()

# Access and print the row with the lowest 'Score' value
row_with_lowest_score = df_list[4].loc[min_index]
print(row_with_lowest_score)

Report_Name    POC_Revenue Opportunities
Page_Name           Opportunity Overview
Title                     Product Slicer
Score                           0.411536
Name: 2, dtype: object
